In [1]:
import torch
import pickle
import numpy as np
import pandas as pd
import plotly.express as px
import torch.optim as optim
from copy import deepcopy
from tqdm import tqdm


from model import LR, NN
from data import FairnessDataset, SyntheticDataset, GermanDataset
from ei_model_dev import FairBatch, Covariance
from ei_effort import Optimal_Effort, PGD_Effort
from ei_utils import *

In [2]:
dataset = SyntheticDataset(seed=0)

In [ ]:
def append_res(d, lamb, alpha, acc, ei, model, model_adv):
    d['lambda'].append(lamb)
    d['alpha'].append(alpha)
    d['accuracy'].append(acc)
    d['loss'].append(1-acc)
    d['ei_disparity'].append(ei)
    d['model'].append(model)
    d['model_adv'].append(model_adv)

In [ ]:
def model_runner(ei_model, dataset, hp, results):
    tau = 0.5
    pga_term = hp['pga_term']
    
    if hp['optimal_effort']:
        effort = Optimal_Effort(hp['delta'])
    else:
        effort = PGD_Effort(hp['delta'])
    
    train_tensors, val_tensors, test_tensors = dataset.tensor(z_blind=hp['z_blind'])
    train_dataset = FairnessDataset(*train_tensors, dataset.imp_feats)
    val_dataset = FairnessDataset(*val_tensors, dataset.imp_feats)
    test_dataset = FairnessDataset(*test_tensors, dataset.imp_feats)
    
    model = LR(num_features=train_dataset.X.shape[1])
    ei_m = ei_model(model, effort, pga_term, tau)
    
    ei_m.train(
        train_dataset, 
        lamb=hp['lambda'],
        lr=hp['learning_rate'],
        alpha=0.,
        batch_size=1024
        )
    
    Y_hat, Y_hat_max = ei_m.predict(test_dataset)
    test_acc, test_ei = model_performance(test_dataset.Y.detach().numpy(), test_dataset.Z.detach().numpy(), Y_hat, Y_hat_max, tau)
    append_res(results, hp['lambda'], 0., test_acc, test_ei, ei_m.model, ei_m.model_adv)
    ei_models = [ei_m.model]
    
    for alpha in hp['alpha']:
        Y_hat_r, Y_hat_max_r = ei_m.predict_r(test_dataset, alpha)
        test_acc_r, test_ei_r = model_performance(test_dataset.Y.detach().numpy(), test_dataset.Z.detach().numpy(), Y_hat_r, Y_hat_max_r, tau)
        append_res(results, hp['lambda'], alpha, test_acc_r, test_ei_r, ei_m.model, ei_m.model_adv)
        ei_models.append(ei_m.model_adv)

In [ ]:
def run_tradeoff(ei_model, dataset, hyper_params):
    hp = hyper_params.copy()
    results = pd.DataFrame()
    
    results = {'lambda': [], 'alpha': [], 'accuracy': [], 'loss': [], 'ei_disparity': [], 'model': [], 'model_adv': []}
    for delta in hyper_params['delta']:
        for lamb in hyper_params['lambda']:
            hp['lambda'] = lamb
            hp['delta'] = delta
            
            model_runner(ei_model, dataset, hp, results)
    
    return results

In [ ]:
torch.manual_seed(0)
# Hyperparameters
hyper_params = {}
hyper_params['delta'] = [0.5]
hyper_params['alpha'] = [0.1, 0.2005, 0.2075, 0.2081, 0.5]
hyper_params['learning_rate'] = 0.01
hyper_params['pga_term'] = 20
hyper_params['z_blind'] = False
hyper_params['optimal_effort'] = True # True only for Synthetic Data

ei_model = FairBatch
# hyper_params['lambda'] = np.linspace(0., 0.25, 10) # FairBatch lambdas
# hyper_params['lambda'] = [0.88] # lambda value that minimizes ei for FairBatch
hyper_params['lambda'] = [0.] # lambda value that minimizes ei for FairBatch

# Run tradeoffs
results = run_tradeoff(ei_model, dataset, hyper_params)
# results['alpha'] = results['alpha'].round(2)
# results_r['alpha'] = results_r['alpha'].round(2)

In [ ]:
res = {
    'lambda': results['lambda'],
    'alpha': results['alpha'],
    'accuracy': results['accuracy'],
    'loss': results['loss'],
    'ei_disparity': results['ei_disparity']
}
res = pd.DataFrame(res)
res

In [ ]:
theta = None
for i in range(6):
    print(f"lambda: {results['lambda'][i]}, alpha: {results['alpha'][i]}")
    model, model_adv = results['model'][i], results['model_adv'][i]
    for module in model.layers:
        if hasattr(module, 'weight'):
            weights_0 = module.weight.data
        print(weights_0)
        if hasattr(module, 'bias'):
            bias_0 = module.bias.data
        theta_0 = torch.cat((weights_0[0], bias_0), 0)
        print(bias_0)
    # print(f'Model: {theta_0}')
        
    # for module in model_adv.layers:
    #     if hasattr(module, 'weight'):
    #         weights = module.weight.data
    #     if hasattr(module, 'bias'):
    #         bias = module.bias.data
    #     theta = torch.cat((weights[0], bias), 0)
    # print(f'Model Adv: {theta}')
    # print(f'alphas: {theta_0-theta}')
    
    # print()

In [ ]:
theta

In [9]:
train_tensors, val_tensors, test_tensors = dataset.tensor(z_blind=False)
train_dataset = FairnessDataset(*train_tensors, dataset.imp_feats)
val_dataset = FairnessDataset(*val_tensors, dataset.imp_feats)
test_dataset = FairnessDataset(*test_tensors, dataset.imp_feats)

model_params = torch.load('../ei_model.pkl')
model = LR(train_dataset.X.shape[1])
model.load_state_dict(model_params)

for module in model.layers:
    if hasattr(module, 'weight'):
        weights = module.weight.data
    if hasattr(module, 'bias'):
        bias = module.bias.data

theta_0 = torch.cat((weights[0], bias), 0)

In [10]:
alpha = 0.2
grid = generate_grid(theta_0.numpy(), alpha, n=5, ord=np.inf)

In [12]:
def grid_search(model, grid, dataset, tau=0.5):
    Y_hat = model(dataset.X).reshape(-1).detach().numpy()
    X_hat_max = Optimal_Effort(tau)(model, dataset, dataset.X)

    accuracies, ei_disparities = [], []
    
    for theta in tqdm(grid[:1]):
        weights, bias = theta[:-1].reshape(1, -1), theta[[-1]]
        # print(weights, bias)
        model_adv = deepcopy(model)
        for module in model_adv.layers:
            if hasattr(module, 'weight'):
                module.weight.data = weights.float()
            if hasattr(module, 'bias'):
                module.bias.data = bias.float()
                
        Y_hat_max = model_adv(X_hat_max)
        
        Y_pred = (Y_hat>tau)*1
        Y_pred_max = (Y_hat_max>tau)*1
        
        n_eyz = {}
        for y_max in [0,1]: 
            for y in [0,1]:
                for z in [0,1]:
                    n_eyz[(y_max,y,z)] = np.sum((Y_pred_max==y_max)*(Y_pred==y)*(dataset.Z==z))
                    
        print(n_eyz)
        z_set = list(set([z for _,_, z in n_eyz.keys()]))
        sum([n_eyz[(1,0,z)]+n_eyz[(0,0,z)] for z in z_set])
    
        
    #     acc, ei_d = model_performance(dataset.Y.detach().numpy(), dataset.Z.detach().numpy(), Y_hat, Y_hat_max.detach().numpy(), 0.5)
    #     accuracies.append(acc)
    #     ei_disparities.append(ei_d)
        
    # max_i = np.argmax(ei_disparities)
    
    # return accuracies[max_i], ei_disparities[max_i], grid[max_i], ei_disparities

In [6]:
# acc, ei_d, theta, ei_d_max = grid_search(model, grid, test_dataset)
grid_search(model, grid, test_dataset)

100%|██████████| 625/625 [00:30<00:00, 20.23it/s]


In [8]:
n_eyz = {}
for y_max in [0,1]: 
    for y in [0,1]:
        for z in [0,1]:
            n_eyz[(y_max,y,z)] = np.sum((Y_pred_max==y_max)*(Y_pred==y)*(Z==z))

tensor([ 0.0218,  0.3095, -0.3942,  0.1345], dtype=torch.float64)

In [11]:
theta_0

tensor([ 0.2218,  0.5095, -0.1942,  0.3345])

In [ ]:
print('EI (train alpha = 0)')
results[['alpha', 'lambda', 'loss', 'ei_disparity']]#.sort_values(['lambda', 'alpha']).set_index(['lambda', 'alpha'])

In [ ]:
print('REI (train alpha = a)')
results_r[['alpha', 'lambda', 'loss', 'ei_disparity']]##.sort_values(['lambda', 'alpha']).set_index(['lambda', 'alpha'])

In [ ]:
# Uncomment this to save the results
# results.to_pickle(f'robust_ei_optimal_lambda_tradeoff_{ei_proxy.lower()}_synthetic_5crossval.pkl')

In [ ]:
# Compute the pareto frontier
results_pareto = pd.DataFrame()
for alpha in results['alpha'].unique():
    test_results_alpha = results[results['alpha'] == alpha]
    mask = pareto_frontier(test_results_alpha['loss_mean'], test_results_alpha['ei_disparity_mean'])
    results_alpha_pareto = test_results_alpha.iloc[mask]
    results_pareto = pd.concat((results_pareto, results_alpha_pareto.sort_values('ei_disparity_mean')))

In [ ]:
fig = px.line(results, x='ei_disparity_mean', y='loss_mean', color='alpha', hover_data='lambda', markers=True)
fig.add_annotation(dict(font=dict(color='black',size=10),
                                        x=0.9,
                                        y=0.99,
                                        showarrow=False,
                                        text='dataset=synthetic',
                                        textangle=0,
                                        xanchor='left',
                                        xref="paper",
                                        yref="paper"))
fig.update_layout(title=dict(text='Fairness vs Loss Tradeoff', x=0.5))
fig.update_traces(marker=dict(size=3))
fig.show()